In [0]:
dbutils.widgets.text("proc_date", "")
proc_date = dbutils.widgets.get("proc_date")

In [0]:
%run ../../../utils/common

In [0]:
dbutils.widgets.text("environment", "", "")
environment = dbutils.widgets.get("environment")

In [0]:
catalog_name = settings[environment]['catalog_name']

In [0]:
create_temp_view_with_clean_columns(
    spark,
    catalog_name,
    'udp_wcm_bronze_sap_bw',
    'sap_bw_wcm_rspmrequest',
    proc_date,
    'temp_sap_rspmrequest'
)

In [0]:
spark.sql(f"""
CREATE TABLE IF NOT EXISTS {catalog_name}.udp_wcm_silver_sap_bw.sap_rspmrequest
(
  hash_id BIGINT,
  request_tsn STRING,
  storage STRING,
  last_operation_type STRING,
  last_process_tsn STRING,
  last_time_stamp STRING,
  records BIGINT,
  creation_end_time STRING,
  request_status STRING,
  request_status_before_deletion STRING,
  request_is_in_process STRING,
  tlogo STRING,
  datatarget STRING,
  syst_date STRING,
  syst_time STRING,
  housekeeping_status STRING,
  uname STRING,
  source STRING,
  proc_date DATE,
  file_creation_ts TIMESTAMP
)
TBLPROPERTIES (
  'DELTA.AUTOOPTIMIZE.OPTIMIZEWRITE' = 'TRUE',
  'DELTA.AUTOOPTIMIZE.AUTOCOMPACT' = 'TRUE'
)""")

In [0]:
spark.sql(f"""
CREATE OR REPLACE TEMP VIEW temp_view AS
SELECT
    request_tsn,
    storage,
    last_operation_type,
    last_process_tsn,
    last_time_stamp,
    records,
    creation_end_time,
    request_status,
    request_status_before_deletion,
    request_is_in_process,
    tlogo,
    datatarget,         
    syst_date,
    syst_time,
    housekeeping_status,
    uname,
    source, 
    proc_date,
    file_creation_ts,
    farm_fingerprint(CONCAT(
        IFNULL(CAST(request_tsn AS STRING), ""),
        IFNULL(CAST(storage AS STRING), ""),
        IFNULL(CAST(last_operation_type AS STRING), ""),
        IFNULL(CAST(last_process_tsn AS STRING), ""),
        IFNULL(CAST(last_time_stamp AS STRING), ""),
        IFNULL(CAST(records AS STRING), ""),
        IFNULL(CAST(creation_end_time AS STRING), ""),
        IFNULL(CAST(request_status AS STRING), ""),
        IFNULL(CAST(request_status_before_deletion AS STRING), ""),
        IFNULL(CAST(request_is_in_process AS STRING), ""),
        IFNULL(CAST(tlogo AS STRING), ""),
        IFNULL(CAST(datatarget AS STRING), ""),
        IFNULL(CAST(syst_date AS STRING), ""),
        IFNULL(CAST(syst_time AS STRING), ""),
        IFNULL(CAST(housekeeping_status AS STRING), ""),
        IFNULL(CAST(uname AS STRING), ""),
        IFNULL(CAST(uname AS STRING), ""),
        IFNULL(CAST(source AS STRING), "")
    )) AS hash_id
FROM temp_sap_rspmrequest
""")

In [0]:
spark.sql(f"""
MERGE INTO {catalog_name}.udp_wcm_silver_sap_bw.sap_rspmrequest main_tbl
USING temp_view temp_tbl
ON main_tbl.hash_id = temp_tbl.hash_id
WHEN NOT MATCHED THEN INSERT *
""")